In [1]:
# Enable autoreload of module
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torchvision import transforms
from networks.mlp_models import MLP3D
from data.neural_field_datasets_shapenet import AllWeights3D, ModelTransform3D, ShapeNetDataset, FlattenTransform3D, ZScore3D, get_neuron_mean_n_std, get_total_mean_n_std


shapeNetData = ShapeNetDataset("./datasets/plane_mlp_weights", transform=AllWeights3D())
means, stds = get_neuron_mean_n_std(shapeNetData)
shapeNetData_normalized = ShapeNetDataset("./datasets/plane_mlp_weights", transform=[AllWeights3D(), ZScore3D(means, 1)])

all_weights = torch.stack([sample[0] for sample in shapeNetData])
all_weights_normalized = torch.stack([sample[0] for sample in shapeNetData_normalized])

/Users/luis/uni/adl4cv/adl4cv/adl4cv/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import wandb
from tqdm import tqdm


# Initialize wandb
wandb.init(project="autoencoder")

# Define the 1D convolutional autoencoder model
class Conv1DAutoencoder(nn.Module):
    def __init__(self):
        super(Conv1DAutoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv1d(128, 64, kernel_size=3, stride=2, padding=1),  # (64, 144)
            nn.ReLU(True),
            nn.Conv1d(64, 32, kernel_size=3, stride=2, padding=1),   # (32, 72)
            nn.ReLU(True),
            nn.Conv1d(32, 16, kernel_size=3, stride=2, padding=1),   # (16, 36)
            nn.ReLU(True),
            nn.Conv1d(16, 8, kernel_size=3, stride=2, padding=1)     # (8, 18)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(8, 16, kernel_size=3, stride=2, padding=1, output_padding=1), # (16, 36)
            nn.ReLU(True),
            nn.ConvTranspose1d(16, 32, kernel_size=3, stride=2, padding=1, output_padding=1), # (32, 72)
            nn.ReLU(True),
            nn.ConvTranspose1d(32, 64, kernel_size=3, stride=2, padding=1, output_padding=1), # (64, 144)
            nn.ReLU(True),
            nn.ConvTranspose1d(64, 128, kernel_size=3, stride=2, padding=1, output_padding=1), # (128, 287)
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Create the model, loss function, and optimizer
model = Conv1DAutoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Load your data
# Assuming your tensor is named images_tensor with shape (3883, 287, 128)
images_tensor = all_weights_normalized  # Example tensor
images_tensor = images_tensor.permute(0, 2, 1)  # Change shape to (3883, 128, 287) for Conv1d
dataset = TensorDataset(images_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    bar = tqdm(dataloader)
    for data in bar:
        img = data[0]
        # Forward pass
        output = model(img)[:, :, :287]
        loss = criterion(output, img)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        bar.set_description(f"Loss: {loss}")
    
    # Log the loss to wandb
    wandb.log({"epoch": epoch, "loss": loss.item()})
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the model
torch.save(model.state_dict(), "conv1d_autoencoder.pth")
wandb.save("conv1d_autoencoder.pth")

wandb.finish()


  0%|          | 0/122 [00:00<?, ?it/s]

In [3]:
all_weights_normalized.shape

torch.Size([3883, 287, 128])

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import wandb
from tqdm import tqdm
# Initialize wandb
wandb.init(project="autoencoder")

# Define the convolutional autoencoder model
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),  # (16, 144, 64)
            nn.ReLU(True),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1), # (32, 72, 32)
            nn.ReLU(True),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1), # (64, 36, 16)
            nn.ReLU(True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1) # (128, 18, 8)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1), # (64, 36, 16)
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),  # (32, 72, 32)
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),  # (16, 144, 64)
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 1, kernel_size=3, stride=2, padding=1, output_padding=1),   # (1, 287, 128)
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Create the model, loss function, and optimizer
model = ConvAutoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Load your data
# Assuming your tensor is named images_tensor with shape (3883, 287, 128)
images_tensor = all_weights_normalized  # Example tensor
images_tensor = images_tensor.unsqueeze(1)  # Add channel dimension (3883, 1, 287, 128)
dataset = TensorDataset(images_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    bar = tqdm(dataloader)
    for data in bar:
        img = data[0]
        # Forward pass
        output = model(img)[:, :, :287, :]
        loss = criterion(output, img)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        bar.set_description(f"Loss: {loss}")
    
    # Log the loss to wandb
    wandb.log({"epoch": epoch, "loss": loss.item()})
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the model
torch.save(model.state_dict(), "conv_autoencoder.pth")
wandb.save("conv_autoencoder.pth")

wandb.finish()


epoch,▁
loss,▁
epoch,0
loss,1.00157


Loss: 0.3099527955055237: 100%|██████████| 122/122 [00:31<00:00,  3.91it/s] 


Epoch [1/50], Loss: 0.3100


Loss: 0.2693328261375427: 100%|██████████| 122/122 [00:34<00:00,  3.59it/s] 


Epoch [2/50], Loss: 0.2693


Loss: 0.3073357343673706:  85%|████████▌ | 104/122 [00:24<00:04,  4.24it/s] 


KeyboardInterrupt: 